In [30]:
import torch
import sklearn
import sklearn.neural_network
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from ipynb.fs.full.Functions import load_train_test_data, load_train_test_oversampled, run_gridsearchCV, load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score 

torch.manual_seed(0)


# Neural network

In [31]:
# Define the network architecture
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
        
    def forward(self, x):
        x = torch.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x


In [32]:
# Load the data
X_train, X_test, y_train, y_test = load_train_test_data()

A snippet of our training data:


,Age,PhysicallyActive,BMI,Sleep,SoundSleep,JunkFood,Stress,BPLevel,Pregnancies,UrinationFreq,BMI_high,highBP_yes,RegularMedicine_yes,Pdiabetes_yes,Alcohol_yes,Gender_Male,Smoking_yes,Family_Diabetes_yes
188,3.0,0.0,-0.514087,-0.775318,-0.293345,0.0,1.0,2.0,-0.416387,0.0,0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
145,0.0,2.0,-1.261602,0.026431,-0.293345,0.0,0.0,1.0,-0.416387,0.0,0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
87,3.0,1.0,-2.009116,-2.378816,-0.838128,0.0,2.0,2.0,-0.416387,1.0,0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
262,3.0,2.0,-0.887844,0.828180,-0.838128,0.0,1.0,1.0,-0.416387,1.0,0,1.0,1.0,0.0,1.0,1.0,0.0,1.0
45,1.0,1.0,-0.887844,-0.775318,0.251438,0.0,1.0,0.0,-0.416387,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


There are 232 entries with 18 columns in our training data.
There are 41 entries with 18 columns in our testing data.


In [33]:
X_train2, X_test2, y_train2, y_test2 = load_train_test_oversampled()

A snippet of our training data:


,Age,PhysicallyActive,BMI,Sleep,SoundSleep,JunkFood,Stress,BPLevel,Pregnancies,UrinationFreq,BMI_high,highBP_yes,RegularMedicine_yes,Pdiabetes_yes,Alcohol_yes,Gender_Male,Smoking_yes,Family_Diabetes_yes
0,0.0,2.0,1.354699,0.828180,-0.838128,0.0,1.0,1.0,-0.416387,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,1.0,0.794063,-0.775318,-1.927694,1.0,3.0,2.0,-0.416387,0.0,1,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,3.0,1.0,0.420306,0.828180,1.341005,0.0,0.0,1.0,-0.416387,1.0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.0,2.0,2.289092,0.828180,-0.838128,0.0,1.0,1.0,-0.416387,1.0,1,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,-0.514087,-1.577067,-0.293345,3.0,1.0,1.0,-0.416387,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


There are 258 entries with 18 columns in our training data.
There are 82 entries with 18 columns in our testing data.


# 1) Neural network with normal preprocessed dataset
## Basic Model

In [34]:
# Convert the data to tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.int64)
y_test = torch.tensor(y_test.values, dtype=torch.int64)

In [35]:
# define the network
X_train.shape[0]
net = Net(n_feature=X_train.shape[1], n_hidden=10, n_output=2)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
loss = torch.nn.CrossEntropyLoss()

batch_size = 100
num_epoch = 20
next_epoch = 1
epoch = 0

In [36]:
for epoch in range(next_epoch, next_epoch+num_epoch):
        
        # Make an entire pass (an 'epoch') over the training data in batch_size chunks
        for i in range(0, len(X_train)):        
            X = X_train[i]     # Slice out a mini-batch of features
            y = y_train[i]     # Slice out a mini-batch of targets
    
            y_pred = net(X)                 # Make predictions (final-layer activations)
            l = loss(y_pred, y)                # Compute loss with respect to predictions
            net.zero_grad()                   # Reset all gradient accumulators to zero (PyTorch thing)
            l.backward()                        # Compute gradient of loss wrt all parameters (backprop!)
            optimizer.step()                    # Use the gradients to take a step with SGD.
            
        print("Epoch %2d: loss on final training batch: %.4f" % (epoch, l.item()))

print("Epoch %2d: loss on test set: %.4f" % (epoch, loss(net(X_test), y_test)))
next_epoch = epoch+1

Epoch  1: loss on final training batch: 0.5101
Epoch  2: loss on final training batch: 0.4755
Epoch  3: loss on final training batch: 0.2560
Epoch  4: loss on final training batch: 0.2919
Epoch  5: loss on final training batch: 0.3011
Epoch  6: loss on final training batch: 0.1699
Epoch  7: loss on final training batch: 0.8593
Epoch  8: loss on final training batch: 0.9908
Epoch  9: loss on final training batch: 0.9971
Epoch 10: loss on final training batch: 0.3695
Epoch 11: loss on final training batch: 0.2506
Epoch 12: loss on final training batch: 0.1976
Epoch 13: loss on final training batch: 0.2532
Epoch 14: loss on final training batch: 0.4406
Epoch 15: loss on final training batch: 0.1412
Epoch 16: loss on final training batch: 0.3214
Epoch 17: loss on final training batch: 0.3475
Epoch 18: loss on final training batch: 0.0947
Epoch 19: loss on final training batch: 0.3414
Epoch 20: loss on final training batch: 0.4145
Epoch 20: loss on test set: 0.6182


In [37]:
# print the accuracy
y_pred = F.softmax(net(X_test)).argmax(dim=1).float()
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))


Accuracy: 0.8292682926829268


C:\Users\Tung\AppData\Local\Temp\ipykernel_25384\2138577276.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_pred = F.softmax(net(X_test)).argmax(dim=1).float()


In [38]:
y_train_pred = F.softmax(net(X_train)).argmax(dim=1).float()
y_test_pred = F.softmax(net(X_test)).argmax(dim=1).float()

functions = [accuracy_score,f1_score,recall_score,precision_score]
for func in functions:
    print(f'\nEvaluation: {func.__name__}')
    v_train =  func(y_train, y_train_pred)
    v_test = func(y_test, y_test_pred)
    print('{0:.2%} for the train data'.format(v_train))
    print('{0:.2%} for the test data'.format(v_test))
    print(accuracy_score(y_train, y_train_pred))


Evaluation: accuracy_score
83.62% for the train data
82.93% for the test data
0.8362068965517241

Evaluation: f1_score
75.32% for the train data
69.57% for the test data
0.8362068965517241

Evaluation: recall_score
77.33% for the train data
61.54% for the test data
0.8362068965517241

Evaluation: precision_score
73.42% for the train data
80.00% for the test data
0.8362068965517241


C:\Users\Tung\AppData\Local\Temp\ipykernel_25384\2602556699.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_train_pred = F.softmax(net(X_train)).argmax(dim=1).float()
C:\Users\Tung\AppData\Local\Temp\ipykernel_25384\2602556699.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_test_pred = F.softmax(net(X_test)).argmax(dim=1).float()


# 2) Neural network with oversampled preprocessed dataset

In [39]:
X_train2, X_test2, y_train2, y_test2 = load_train_test_oversampled()

A snippet of our training data:


,Age,PhysicallyActive,BMI,Sleep,SoundSleep,JunkFood,Stress,BPLevel,Pregnancies,UrinationFreq,BMI_high,highBP_yes,RegularMedicine_yes,Pdiabetes_yes,Alcohol_yes,Gender_Male,Smoking_yes,Family_Diabetes_yes
0,0.0,2.0,1.354699,0.828180,-0.838128,0.0,1.0,1.0,-0.416387,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,1.0,0.794063,-0.775318,-1.927694,1.0,3.0,2.0,-0.416387,0.0,1,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,3.0,1.0,0.420306,0.828180,1.341005,0.0,0.0,1.0,-0.416387,1.0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.0,2.0,2.289092,0.828180,-0.838128,0.0,1.0,1.0,-0.416387,1.0,1,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,-0.514087,-1.577067,-0.293345,3.0,1.0,1.0,-0.416387,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


There are 258 entries with 18 columns in our training data.
There are 82 entries with 18 columns in our testing data.


In [40]:
X_train2 = torch.tensor(X_train2.values, dtype=torch.float32)
X_test2 = torch.tensor(X_test2.values, dtype=torch.float32)
y_train2 = torch.tensor(y_train2.values, dtype=torch.int64)
y_test2 = torch.tensor(y_test2.values, dtype=torch.int64)

In [41]:
X_train2.shape[0]
net = Net(n_feature=X_train2.shape[1], n_hidden=10, n_output=2)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
loss = torch.nn.CrossEntropyLoss()

batch_size = 100
num_epoch = 20
next_epoch = 1
epoch = 0

In [42]:
# define the grid for the hyperparameter search
for epoch in range(next_epoch, next_epoch+num_epoch):
        
        # Make an entire pass (an 'epoch') over the training data in batch_size chunks
        for i in range(0, len(X_train2)):        
            X_2 = X_train2[i]     # Slice out a mini-batch of features
            y_2 = y_train2[i]     # Slice out a mini-batch of targets
    
            y_pred_2 = net(X_2)                 # Make predictions (final-layer activations)
            l = loss(y_pred_2, y_2)                # Compute loss with respect to predictions
            net.zero_grad()                   # Reset all gradient accumulators to zero (PyTorch thing)
            l.backward()                        # Compute gradient of loss wrt all parameters (backprop!)
            optimizer.step()                    # Use the gradients to take a step with SGD.
            
        print("Epoch %2d: loss on final training batch: %.4f" % (epoch, l.item()))

print("Epoch %2d: loss on test set: %.4f" % (epoch, loss(net(X_test2), y_test2)))
next_epoch = epoch+1

Epoch  1: loss on final training batch: 0.0000
Epoch  2: loss on final training batch: 0.0000
Epoch  3: loss on final training batch: 0.0000
Epoch  4: loss on final training batch: 0.0000
Epoch  5: loss on final training batch: 0.0000
Epoch  6: loss on final training batch: 0.0000
Epoch  7: loss on final training batch: 0.0000
Epoch  8: loss on final training batch: 0.0000
Epoch  9: loss on final training batch: 0.0001
Epoch 10: loss on final training batch: 0.0001
Epoch 11: loss on final training batch: 0.0003
Epoch 12: loss on final training batch: 0.0000
Epoch 13: loss on final training batch: 0.0001
Epoch 14: loss on final training batch: 0.0001
Epoch 15: loss on final training batch: 0.0002
Epoch 16: loss on final training batch: 0.0020
Epoch 17: loss on final training batch: 0.0014
Epoch 18: loss on final training batch: 0.0011
Epoch 19: loss on final training batch: 0.0020
Epoch 20: loss on final training batch: 0.0065
Epoch 20: loss on test set: 1.3334


In [43]:
y_train_pred2 = F.softmax(net(X_train2)).argmax(dim=1).float()
y_test_pred2 = F.softmax(net(X_test2)).argmax(dim=1).float()

functions = [accuracy_score,f1_score,recall_score,precision_score]
for func in functions:
    print(f'\nEvaluation: {func.__name__}')
    v_train =  func(y_train2, y_train_pred2)
    v_test = func(y_test2, y_test_pred2)
    print('{0:.2%} for the train data'.format(v_train))
    print('{0:.2%} for the test data'.format(v_test))
    print(accuracy_score(y_train2, y_train_pred2))


Evaluation: accuracy_score
88.76% for the train data
65.85% for the test data
0.8875968992248062

Evaluation: f1_score
89.90% for the train data
60.00% for the test data
0.8875968992248062

Evaluation: recall_score
100.00% for the train data
80.77% for the test data
0.8875968992248062

Evaluation: precision_score
81.65% for the train data
47.73% for the test data
0.8875968992248062


C:\Users\Tung\AppData\Local\Temp\ipykernel_25384\2836192748.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_train_pred2 = F.softmax(net(X_train2)).argmax(dim=1).float()
C:\Users\Tung\AppData\Local\Temp\ipykernel_25384\2836192748.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_test_pred2 = F.softmax(net(X_test2)).argmax(dim=1).float()
